In [30]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate, Conv1D, Embedding, SpatialDropout1D, GlobalMaxPooling1D, Dense, Input, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
NUM_WORDS = 10000
MAX_LENGTH_REVIEW = 400
N_DIM = 64
BATCH_SIZE = 128
DROP_EMBED = dropout = 0.2

n_conv_1 = n_conv_2 = n_conv_3 = 256 
k_conv_1 = 3
k_conv_2 = 2
k_conv_3 = 4

n_dense = 256

In [42]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS, maxlen=MAX_LENGTH_REVIEW)

x_train_pad = pad_sequences(x_train, maxlen=MAX_LENGTH_REVIEW)
x_test_pad = pad_sequences(x_test, maxlen=MAX_LENGTH_REVIEW)

In [43]:
input = Input(shape=(MAX_LENGTH_REVIEW,), dtype='int16', name='input')
embed_layer = Embedding(NUM_WORDS, N_DIM, name='embed_layer')(input)
drop_layer = SpatialDropout1D(DROP_EMBED)(embed_layer)

conv_1_l = Conv1D(n_conv_1, k_conv_1, name='conv_1_l')(drop_layer)
max_p_1_l = GlobalMaxPooling1D(name='max_p_1_l')(conv_1_l)

conv_2_l = Conv1D(n_conv_2, k_conv_2, name='conv_2_l')(drop_layer)
max_p_2_l = GlobalMaxPooling1D(name='max_p_2_l')(conv_2_l)

conv_3_l = Conv1D(n_conv_3, k_conv_3, name='conv_3_l')(drop_layer)
max_p_3_l = GlobalMaxPooling1D(name='max_p_3_l')(conv_3_l)

concat = concatenate([max_p_1_l, max_p_2_l, max_p_3_l])

dense_1 = Dense(n_dense, activation='relu', name='dense_1_l')(concat)
drop_1 = Dropout(dropout, name='drop_1')(dense_1)

dense_2 = Dense(n_dense, activation='relu', name='dense_2_l')(drop_1)
drop_2 = Dropout(dropout, name='drop_2')(dense_2)

predict = Dense(1, activation='sigmoid', name='output')(drop_2)

model = Model(input, predict)

model.summary()

Model: "functional_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 400)]        0                                            
__________________________________________________________________________________________________
embed_layer (Embedding)         (None, 400, 64)      640000      input[0][0]                      
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 400, 64)      0           embed_layer[0][0]                
__________________________________________________________________________________________________
conv_1_l (Conv1D)               (None, 398, 256)     49408       spatial_dropout1d_3[0][0]        
______________________________________________________________________________________

In [44]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC', 'acc'])
model.fit(x_train_pad, y_train, validation_data=(x_test_pad, y_test), epochs=5, verbose=2)

Epoch 1/5
672/672 - 60s - loss: 0.4178 - auc: 0.8865 - acc: 0.7873 - val_loss: 0.2636 - val_auc: 0.9587 - val_acc: 0.8911
Epoch 2/5
672/672 - 56s - loss: 0.2122 - auc: 0.9719 - acc: 0.9186 - val_loss: 0.2606 - val_auc: 0.9620 - val_acc: 0.8966
Epoch 3/5
672/672 - 56s - loss: 0.1223 - auc: 0.9897 - acc: 0.9562 - val_loss: 0.2809 - val_auc: 0.9586 - val_acc: 0.8948
Epoch 4/5
672/672 - 59s - loss: 0.0635 - auc: 0.9967 - acc: 0.9793 - val_loss: 0.4196 - val_auc: 0.9509 - val_acc: 0.8716
Epoch 5/5
672/672 - 59s - loss: 0.0393 - auc: 0.9986 - acc: 0.9873 - val_loss: 0.4398 - val_auc: 0.9468 - val_acc: 0.8853
